In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
df = pd.read_csv("air_quality_dataset.csv") 
print(df.info()) #check info

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2164 entries, 0 to 2163
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Rank     2164 non-null   int64  
 1   City     2164 non-null   object 
 2   Country  2164 non-null   object 
 3   2023     2164 non-null   float64
 4   Jan      2164 non-null   object 
 5   Feb      2164 non-null   object 
 6   Mar      2164 non-null   object 
 7   Apr      2164 non-null   object 
 8   May      2164 non-null   object 
 9   Jun      2164 non-null   object 
 10  Jul      2164 non-null   object 
 11  Aug      2164 non-null   object 
 12  Sep      2164 non-null   object 
 13  Oct      2164 non-null   object 
 14  Nov      2164 non-null   object 
 15  Dec      2164 non-null   object 
dtypes: float64(1), int64(1), object(14)
memory usage: 270.6+ KB
None


Data types should be similar so we need to change to float

In [4]:
for col in df.columns[-12:]:
    df[col] = pd.to_numeric(df[col],errors="coerce")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2164 entries, 0 to 2163
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Rank     2164 non-null   int64  
 1   City     2164 non-null   object 
 2   Country  2164 non-null   object 
 3   2023     2164 non-null   float64
 4   Jan      2077 non-null   float64
 5   Feb      2098 non-null   float64
 6   Mar      2131 non-null   float64
 7   Apr      2155 non-null   float64
 8   May      2106 non-null   float64
 9   Jun      2158 non-null   float64
 10  Jul      2160 non-null   float64
 11  Aug      2153 non-null   float64
 12  Sep      2153 non-null   float64
 13  Oct      2133 non-null   float64
 14  Nov      2073 non-null   float64
 15  Dec      2113 non-null   float64
dtypes: float64(13), int64(1), object(2)
memory usage: 270.6+ KB


In [5]:
## check missing values by columns
df.isnull().sum(axis=0).sort_values(ascending=False)

Nov        91
Jan        87
Feb        66
May        58
Dec        51
Mar        33
Oct        31
Aug        11
Sep        11
Apr         9
Jun         6
Jul         4
Rank        0
City        0
Country     0
2023        0
dtype: int64

In [6]:
np.sum(df.isnull().sum(axis=1) > 0)

np.int64(270)

In [ ]:
print(df.isnull().sum().sum())

458


270 out of 2164 rows (12.48%) contain at least one missing value. And total amount is 458 missed value

In [ ]:
df.iloc[:, -12:] = df.iloc[:, -12:].apply(lambda col: col.fillna(col.mean()), axis=0)#adding mean to missing valuse
sum(df.isnull().sum())#sum of missing values


0

In [18]:
df.head(20)

,Rank,City,Country,2023,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1,Begusarai,India,118.9,31.2,235.3,156.8,113.0,109.3,99.0,63.800000,61.8,71.5,61.8,210.5,285.0
1,2,Guwahati,India,105.4,220.2,168.1,129.2,112.2,69.5,51.3,46.600000,60.2,76.7,76.4,126.9,128.0
2,3,Delhi,India,102.1,171.8,114.3,77.4,71.0,67.4,42.9,35.300000,34.8,39.7,106.3,255.1,210.0
3,4,Mullanpur,India,100.4,106.3,123.7,78.1,56.6,53.4,53.9,63.200000,59.7,59.6,110.4,253.0,201.4
4,5,Lahore,Pakistan,99.5,143.2,117.3,73.8,52.9,52.4,46.4,39.800000,42.2,53.8,125.9,251.0,197.5
5,6,New Delhi,India,92.7,162.6,98.2,67.1,59.0,57.7,40.1,31.700000,35.0,38.0,94.7,234.7,193.8
6,7,Siwan,India,90.6,223.6,167.5,108.3,71.7,59.8,48.9,35.700000,30.4,54.7,48.9,136.3,77.6
7,8,Saharsa,India,89.4,202.0,147.1,108.8,88.8,60.3,43.6,16.700000,24.3,33.6,41.8,115.8,167.8
8,9,Goshaingaon,India,89.3,205.3,117.5,63.8,63.3,60.7,39.2,27.300000,38.3,33.9,81.1,152.0,156.1
9,10,Katihar,India,88.8,224.1,113.3,94.0,74.1,49.9,34.7,17.700000,27.6,33.9,63.9,134.1,180.5


In [21]:
df.duplicated().sum()
#checking for duplicates
df.describe()


,Rank,2023,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
count,2164.000000,2164.000000,2164.000000,2164.000000,2164.000000,2164.000000,2164.000000,2164.000000,2164.000000,2164.000000,2164.000000,2164.000000,2164.000000,2164.000000
mean,1082.500000,23.196811,34.433462,30.304147,29.213468,27.340510,19.556363,15.804727,14.068426,14.732699,15.476777,21.324051,27.730005,30.366872
std,624.837312,16.075976,33.013824,24.732518,22.827612,22.294342,11.800626,9.490754,8.553461,10.188459,11.128637,17.482558,30.138818,29.061788
min,1.000000,1.500000,0.900000,0.800000,1.400000,1.400000,1.500000,1.500000,1.200000,1.500000,1.100000,1.000000,0.000000,1.000000
25%,541.750000,10.100000,10.300000,9.900000,12.175000,12.700000,10.200000,9.700000,9.100000,7.500000,7.700000,8.400000,10.300000,9.800000
50%,1082.500000,19.100000,25.200000,24.500000,23.200000,20.400000,17.850000,12.700000,11.500000,11.900000,11.900000,16.100000,18.700000,21.300000
75%,1623.250000,31.200000,44.500000,42.500000,39.450000,35.225000,24.900000,19.400000,16.700000,18.700000,20.100000,27.800000,32.100000,40.325000
max,2164.000000,118.900000,225.300000,235.300000,252.500000,232.000000,109.300000,99.000000,88.900000,99.600000,155.400000,156.000000,255.100000,285.000000
